Calculating covariant derivatives:

In [16]:
import numpy as np
from sympy import symbols, cos, sin, sqrt, tanh, atanh, atan2
from sympy.diffgeom import (Manifold, Patch, CoordSystem, BaseCovarDerivativeOp, CovarDerivativeOp, TensorProduct as TP,
metric_to_Christoffel_2nd, metric_to_Christoffel_1st, twoform_to_matrix)
from sympy.utilities import lambdify

In [2]:
m = Manifold('M', 2)
p = Patch('P', m)

In [3]:
phi, chi = symbols('phi chi', real=True)
r, psi, theta = symbols('r psi theta', real=True, nonnegative=True)
alpha, mphi, R = symbols('alpha m_phi R', real=True, nonnegative=True)

In [4]:
relation_dict = {('PhiChi', 'Pol'): [(phi, chi), (sqrt(phi**2 + chi**2), atan2(chi, phi))],
                ('Pol', 'PhiChi'): [(r, theta), (r*cos(theta), r*sin(theta))],
                ('Pol', 'Canoni'): [(r, theta), (sqrt(6*alpha)*atanh(r), theta)],
                ('Canoni', 'Pol'): [(psi, theta), (tanh(psi/sqrt(6*alpha)), theta)]}

In [5]:
R2_phichi = CoordSystem('PhiChi', p, (phi, chi), relation_dict)
R2_polar = CoordSystem('Pol', p, (r, theta), relation_dict)
R2_canonical = CoordSystem('Canoni', p, (psi, theta), relation_dict)

In [6]:
fphi, fchi = R2_phichi.base_scalars()
e_phi, e_chi = R2_phichi.base_vectors()
dphi, dchi = R2_phichi.base_oneforms()

In [7]:
metric = 6*alpha / (1 - fphi**2 - fchi**2)**2 * (TP(dphi, dphi) + TP(dchi, dchi))

In [20]:
lambdify([phi, chi], twoform_to_matrix(metric).subs(alpha, 1/600))(1,1)

array([[0.01, 0.  ],
       [0.  , 0.01]])

In [13]:
gamma = metric_to_Christoffel_2nd(metric)
gamma.simplify()

[[[-2*phi/(phi**2 + chi**2 - 1), -2*chi/(phi**2 + chi**2 - 1)], [-2*chi/(phi**2 + chi**2 - 1), 2*phi/(phi**2 + chi**2 - 1)]], [[2*chi/(phi**2 + chi**2 - 1), -2*phi/(phi**2 + chi**2 - 1)], [-2*phi/(phi**2 + chi**2 - 1), -2*chi/(phi**2 + chi**2 - 1)]]]

In [14]:
gamma = metric_to_Christoffel_1st(metric)
gamma.simplify()

[[[-12*alpha*(phi**2 + chi**2 - 1)*phi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2, -12*alpha*(phi**2 + chi**2 - 1)*chi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2], [-12*alpha*(phi**2 + chi**2 - 1)*chi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2, 12*alpha*(phi**2 + chi**2 - 1)*phi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2]], [[12*alpha*(phi**2 + chi**2 - 1)*chi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2, -12*alpha*(phi**2 + chi**2 - 1)*phi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2], [-12*alpha*(phi**2 + chi**2 - 1)*phi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2, -12*alpha*(phi**2 + chi**2 - 1)*chi/(phi**4 + 2*phi**2*chi**2 - 2*phi**2 + chi**4 - 2*chi**2 + 1)**2]]]

In [15]:
def gamma_to_np(gamma, coord_symbs):
    return lambdify(coord_symbs, gamma)

In [16]:
gamma_to_np(gamma, [phi, chi])(1, 1)

[[[-1.0, -1.0], [-1.0, 1.0]], [[1.0, -1.0], [-1.0, -1.0]]]

In [28]:
cvds = [BaseCovarDerivativeOp(R2_phichi, 0, gamma), BaseCovarDerivativeOp(R2_phichi, 1, gamma)]
cvds

[BaseCovarDerivativeOp(PhiChi, 0, [[[phi/(-phi**2 - chi**2 + 1), chi/(-phi**2 - chi**2 + 1)], [chi/(-phi**2 - chi**2 + 1), -phi/(-phi**2 - chi**2 + 1)]], [[-chi/(-phi**2 - chi**2 + 1), phi/(-phi**2 - chi**2 + 1)], [phi/(-phi**2 - chi**2 + 1), chi/(-phi**2 - chi**2 + 1)]]]),
 BaseCovarDerivativeOp(PhiChi, 1, [[[phi/(-phi**2 - chi**2 + 1), chi/(-phi**2 - chi**2 + 1)], [chi/(-phi**2 - chi**2 + 1), -phi/(-phi**2 - chi**2 + 1)]], [[-chi/(-phi**2 - chi**2 + 1), phi/(-phi**2 - chi**2 + 1)], [phi/(-phi**2 - chi**2 + 1), chi/(-phi**2 - chi**2 + 1)]]])]

Return covariant derivatives of a vector as a Python function:

In [138]:
def nabla_vec(coords, covar_derivs, vector):
    cvds_expr = [[covar_derivs[ii](vector).rcall(coords[jj]) for jj in range(len(coords))] for ii in range(len(covar_derivs))]
    return lambdify([phi, chi], cvds_expr)

In [141]:
nabla_vec([fphi, fchi], cvds, e_phi)(1, 1)

[[-1.0, 1.0], [-1.0, -1.0]]

In [135]:
np.gradient(np.array([[1,2], [3,4]]), 0.1, axis=0)

array([[20., 20.],
       [20., 20.]])

In [206]:
def nabla_i_vec(vectors: np.ndarray, gamma_vals: np.ndarray, i: int, dN: float) -> np.ndarray:
    return np.gradient(vectors, dN, axis=0)[i] \
        + np.array([[gamma_vals[jj, 0, i, 0] * vectors[jj, 0] + gamma_vals[jj, 0, i, 1] * vectors[jj, 1],
                    gamma_vals[jj, 1, i, 0] * vectors[jj, 0] + gamma_vals[jj, 1, i, 1] * vectors[jj, 1]]
                    for jj in range(len(vectors))])

def nabla_vec(vectors: np.ndarray, gamma_vals: np.ndarray, dN: float) -> np.ndarray:
    return np.append(nabla_i_vec(vectors, gamma_vals, 0, dN),
                    nabla_i_vec(vectors, gamma_vals, 1, dN),
                    axis=1).reshape(len(vectors), 2, 2)

In [207]:
gamma_vals = np.array(gamma_to_np(gamma, [phi, chi])(1, 1))
gamma_vals = np.array([gamma_vals, gamma_vals, gamma_vals])
gamma_vals

array([[[[-1., -1.],
         [-1.,  1.]],

        [[ 1., -1.],
         [-1., -1.]]],


       [[[-1., -1.],
         [-1.,  1.]],

        [[ 1., -1.],
         [-1., -1.]]],


       [[[-1., -1.],
         [-1.,  1.]],

        [[ 1., -1.],
         [-1., -1.]]]])

In [208]:
vectors = np.array([[1, 0], [1, 0], [1, 0]])
nabla_i_vec(vectors, gamma_vals, 0, 1)

array([[-1.,  1.],
       [-1.,  1.],
       [-1.,  1.]])

In [209]:
nabla_i_vec(vectors, gamma_vals, 1, 1)

array([[-1., -1.],
       [-1., -1.],
       [-1., -1.]])

In [210]:
nabla_vec(vectors, gamma_vals, 1.)

array([[[-1.,  1.],
        [-1., -1.]],

       [[-1.,  1.],
        [-1., -1.]],

       [[-1.,  1.],
        [-1., -1.]]])

Return covariant derivative of a vector with respect to the number of e-folds:

In [191]:
def DN_vector(nablavec: np.ndarray, phi_primes: np.ndarray) -> np.ndarray:
    return np.array([phi_primes[ii, 0]*nablavec[ii, 0] + phi_primes[ii, 1] * nablavec[ii, 1] for ii in range(len(phi_primes))])

In [192]:
phi_primes = np.array([[1, 0], [1, 0], [1, 0]])

In [193]:
DN_vector(nabla_vec(vectors, gamma_vals, 1), phi_primes)

array([[-1.,  1.],
       [-1.,  1.],
       [-1.,  1.]])

In [118]:
nablavec = np.array(nabla_vec([fphi, fchi], cvds, fphi*e_chi)(1, 1))
nablavec

array([[-1.,  0.],
       [ 1., -1.]])

In [119]:
phi_primes = np.array([1., 0])
DN_vector(nablavec, phi_primes)

array([[-1.,  0.],
       [ 0., -0.]])

In [120]:
def array_to_sympy(v: np.ndarray, base_vs):
    return sum([v[ii] * base_vs[ii] for ii in range(len(v))])

In [121]:
array_to_sympy([1, 0], [e_phi, e_chi])

e_phi